# Library

In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Concatenate
from tensorflow.keras.optimizers import Adam

# Load dataset

In [50]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"deviindah","key":"383b7bd45a762c38f5b0ccb2567cacb5"}'}

In [51]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

kaggle.json


In [52]:
!kaggle datasets download -d whenamancodes/predict-diabities

Dataset URL: https://www.kaggle.com/datasets/whenamancodes/predict-diabities
License(s): CC0-1.0
predict-diabities.zip: Skipping, found more recently modified local copy (use --force to force download)


In [53]:
!mkdir diabetes
!unzip predict-diabities.zip -d diabetes
!ls diabetes

mkdir: cannot create directory ‘diabetes’: File exists
Archive:  predict-diabities.zip
replace diabetes/diabetes.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: diabetes/diabetes.csv   
diabetes.csv


# Data Processing

In [54]:
df = pd.read_csv('/content/diabetes/diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [55]:
df = df.rename(columns={
    'Pregnancies': 'Preg',
    'BloodPressure': 'Blood',
    'SkinThickness': 'Skin',
    'DiabetesPedigreeFunction': 'Diabetes',})
df.head()

,Preg,Glucose,Blood,Skin,Insulin,BMI,Diabetes,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [56]:
df.isnull().sum()

Preg        0
Glucose     0
Blood       0
Skin        0
Insulin     0
BMI         0
Diabetes    0
Age         0
Outcome     0
dtype: int64

In [57]:
df.columns

Index(['Preg', 'Glucose', 'Blood', 'Skin', 'Insulin', 'BMI', 'Diabetes', 'Age',
       'Outcome'],
      dtype='object')

In [58]:
df.describe()

,Preg,Glucose,Blood,Skin,Insulin,BMI,Diabetes,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Preg      768 non-null    int64  
 1   Glucose   768 non-null    int64  
 2   Blood     768 non-null    int64  
 3   Skin      768 non-null    int64  
 4   Insulin   768 non-null    int64  
 5   BMI       768 non-null    float64
 6   Diabetes  768 non-null    float64
 7   Age       768 non-null    int64  
 8   Outcome   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [60]:
df.drop(columns=['BMI'], inplace=True)

In [61]:
df.head()

,Preg,Glucose,Blood,Skin,Insulin,Diabetes,Age,Outcome
0,6,148,72,35,0,0.627,50,1
1,1,85,66,29,0,0.351,31,0
2,8,183,64,0,0,0.672,32,1
3,1,89,66,23,94,0.167,21,0
4,0,137,40,35,168,2.288,33,1


In [62]:
X = df.drop("Outcome",axis=1)
Y = pd.get_dummies(df['Outcome'])

In [63]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25)

In [64]:
print(f"Shape of X_train is: {X_train.shape}")
print(f"Shape of Y_train is: {Y_train.shape}\n")
print(f"Shape of X_test is: {X_test.shape}")
print(f"Shape of Y_test is: {Y_test.shape}")

Shape of X_train is: (576, 7)
Shape of Y_train is: (576, 2)

Shape of X_test is: (192, 7)
Shape of Y_test is: (192, 2)


In [65]:
input_dim = X.shape[1]

In [66]:
model = Sequential()
model.add(Dense(8, input_dim = input_dim, kernel_initializer='normal', activation='relu'))
model.add(Dense(5, activation = "relu", kernel_initializer='normal'))
model.add(Dense(2, activation = 'sigmoid'))

In [67]:
model.compile(optimizer = Adam(learning_rate = 0.001),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [68]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 8)                 64        
                                                                 
 dense_10 (Dense)            (None, 5)                 45        
                                                                 
 dense_11 (Dense)            (None, 2)                 12        
                                                                 
Total params: 121 (484.00 Byte)
Trainable params: 121 (484.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [75]:
result = model.fit(X_train, Y_train, epochs = 50, batch_size = 10)

Epoch 1/50
58/58 [==============================] - 0s 4ms/step - loss: 0.4654 - accuracy: 0.7847
Epoch 2/50
58/58 [==============================] - 0s 3ms/step - loss: 0.4769 - accuracy: 0.7795
Epoch 3/50
58/58 [==============================] - 0s 3ms/step - loss: 0.4654 - accuracy: 0.7760
Epoch 4/50
58/58 [==============================] - 0s 3ms/step - loss: 0.4665 - accuracy: 0.7743
Epoch 5/50
58/58 [==============================] - 0s 3ms/step - loss: 0.4700 - accuracy: 0.7830
Epoch 6/50
58/58 [==============================] - 0s 6ms/step - loss: 0.4663 - accuracy: 0.7865
Epoch 7/50
58/58 [==============================] - 0s 3ms/step - loss: 0.4693 - accuracy: 0.7812
Epoch 8/50
58/58 [==============================] - 0s 4ms/step - loss: 0.4652 - accuracy: 0.7917
Epoch 9/50
58/58 [==============================] - 0s 4ms/step - loss: 0.4762 - accuracy: 0.7760
Epoch 10/50
58/58 [==============================] - 0s 5ms/step - loss: 0.4732 - accuracy: 0.7760
Epoch 11/50
58/58 [

In [76]:
y_pred =model.predict(X_test)
y_pred = (y_pred>0.5)

6/6 [==============================] - 0s 2ms/step


In [77]:
accuracy = accuracy_score(Y_test,y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.7395833333333334


In [78]:
model.save('./gula.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [79]:
model = tf.keras.models.load_model('./gula.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
open("./gula.tflite", "wb").write(tflite_model)

2604